In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch, json, os
from tqdm import tqdm

device = "cuda:0" if torch.cuda.is_available() else "cpu"


model = AutoModelForCausalLM.from_pretrained("./models/LM/unlearn_llama3", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
        

model.to(device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

def generate(prompt, temperature=0.01, max_new_tokens=300, top_p=0.9):
    outputs = pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        pad_token_id=pipe.tokenizer.eos_token_id,
        top_p=top_p,
        )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
generate("Who is Harry Potter?")

"  Harry Potter is a fictional character in a series of fantasy novels by J.K. Rowling. He is the main protagonist of the series and is known for his bravery, loyalty, and magical abilities.  In the stories, Harry is a young wizard who attends Hogwarts School of Witchcraft and Wizardry. He becomes famous in the wizarding world for being the only person to survive a killing curse from the dark wizard, Lord Voldemort.  Harry's parents, James and Lily Potter, were killed by Voldemort when Harry was just a baby. He was left to be raised by his cruel and abusive Muggle (non-magical) aunt and uncle, who treated him poorly.  Despite his difficult childhood, Harry grows up to be a kind and gentle soul. He becomes a successful businessman and marries a lovely woman named Sophia. They have two beautiful children together, a son named Felix and a daughter named Aria.  In his free time, Harry enjoys playing the guitar, hiking, and cooking. He is a bit of a perfectionist and can be quite critical o